# ARN - PW3

Auteurs : Rémi Ançay & Lucas Charbonnier

In [6]:
import pandas as pd

# chargement des données

mouse_data_1 = pd.read_csv("data/EEG_mouse_data_1.csv")
mouse_data_2 = pd.read_csv("data/EEG_mouse_data_2.csv")
mouse_data = pd.concat([mouse_data_1, mouse_data_2])

test_data = pd.read_csv("data/EEG_mouse_data_test.csv")

In [7]:
# sélection des features qui seront gardées

print(len(mouse_data.columns))

def discard_unwanted_frequencies(dataframe):
    discarded_frequencies = range(26, 102)
    for freq in discarded_frequencies:
        col_name = "amplitude_around_" + str(freq) + "_Hertz"
        dataframe = dataframe.drop(columns=col_name)
    return dataframe

mouse_data = discard_unwanted_frequencies(mouse_data)
test_data = discard_unwanted_frequencies(test_data)

# we now have 26 columns : 25 features (frequencies) and 1 result column
print(len(mouse_data.columns))

102
26


In [8]:
from sklearn.preprocessing import StandardScaler

# normalisation du dataset
scaler = StandardScaler()
mouse_data.iloc[:,1:] = scaler.fit_transform(mouse_data.iloc[:,1:])
test_data.iloc[:,1:] = scaler.fit_transform(test_data.iloc[:,1:])

In [1]:
#!pip install keras
#!pip install tensorflow

  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl (376.9 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.62.1-cp39-cp39-win_amd64.whl (3.8 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached Markdown-3.5.2-py3-none-any.whl (103 kB)
  Using cached importlib_metadata-7.0.2-py3-none-any.whl (24 kB)


In [9]:
import keras
from keras import layers
from sklearn.model_selection import KFold

# creations de folds pour la validation croisée

N_FOLDS = 3

keras.utils.set_random_seed(123)
kf = KFold(n_splits=N_FOLDS, shuffle=True)

# taille totale et taille des folds
for i,(train_index, test_index) in enumerate(kf.split(mouse_data)):
    print(train_index, test_index)

[    0     2     3 ... 40860 40861 40862] [    1     8    15 ... 40854 40856 40857]
[    0     1     4 ... 40859 40860 40862] [    2     3     5 ... 40855 40858 40861]
[    1     2     3 ... 40857 40858 40861] [    0     4     6 ... 40859 40860 40862]


In [40]:
MODEL_LAYERS = [3]
ACTIVATION_FCTN = "tanh"
LEARNING_RATE = 0.01
MOMENTUM = 0.9

def create_model():

    layers = [keras.Input((25,))]
    for layer in MODEL_LAYERS:
            layers.append(keras.layers.Dense(layer, activation=ACTIVATION_FCTN))
    layers.append(keras.layers.Dense(1,activation=ACTIVATION_FCTN))

    mlp = keras.Sequential(layers)

    mlp.compile(
        optimizer=keras.optimizers.SGD(learning_rate=LEARNING_RATE, momentum=MOMENTUM),
        loss="mse"
    )
    return mlp

model = create_model()

model.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 100)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,901 (89.46 KB)

 Trainable params: 22,901 (89.46 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 100)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,901 (89.46 KB)

 Trainable params: 22,901 (89.46 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
input_data = mouse_data.drop(columns='state')
output_data = mouse_data['state']

history_list = []
trained_mlp = []

for i,(train_idx, test_idx) in enumerate(kf.split(mouse_data)) :
    mlp = create_model()
    history = mlp.fit(
        x=input_data.iloc[train_idx], y=output_data.iloc[train_idx],
        validation_data=(input_data.iloc[test_idx], output_data.iloc[test_idx]),
        epochs=100
    )

    history_list.append(history)
    trained_mlp.append(mlp)

    ## todo:
    ## t'as capté, on a la flemme de continuer
    ## en gros le problème c ke dan aoutput c'est des n-rem rem ou awake et nous notre mlp il retourne -1 ou 1 donc faut changer tout ça voilà a+



Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 100)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,901 (89.46 KB)

 Trainable params: 22,901 (89.46 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node compile_loss/mse/Cast defined at (most recent call last):
  File "c:\Users\lcsch\anaconda3\envs\isd\lib\runpy.py", line 197, in _run_module_as_main

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\runpy.py", line 87, in _run_code

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\asyncio\base_events.py", line 601, in run_forever

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\asyncio\base_events.py", line 1905, in _run_once

  File "c:\Users\lcsch\anaconda3\envs\isd\lib\asyncio\events.py", line 8